# HW 4 Работа с текстом

## Получение датасета 

In [101]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для времени

import warnings
warnings.filterwarnings("ignore")


In [102]:
#!pip install pytest-cov
#!pip install pytest-filter-subpackage
#!pip install fake_useragent

In [103]:
# подгрузим один из методов этой библиотеки
from fake_useragent import UserAgent

In [104]:
UserAgent().chrome

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'

In [105]:
from bs4 import BeautifulSoup

Собирем данные об отзывах о фильмах на сайте  https://irecommend.ru/

In [107]:
page_link = 'https://irecommend.ru/content/passazhiry-passengers-0'

In [109]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [110]:
html = response.content

In [111]:
soup = BeautifulSoup(html, 'html.parser')

Ищем информацию с комментарием к фильму и оценкой в виде звезд (от 1 до 5)

In [112]:
views = soup.findAll(attrs={'class':'smTeaser'})[1]
views

<div class="smTeaser woProduct plate reviews-list-item teaser-item" data-nid="9369899" data-photos-count="29" data-product-id="3601788" data-type="1" data-vid="16376367">
<div class="authorAndPhoto">
<div class="authorSpace">
<div class="half1">
<div class="authorPhoto"><div class="picture">
<a href="/users/nadezhda4557785" title="Информация о пользователе."><img alt="Надежда4557785 аватар" class="lazy lazy-loader" data-original="https://cdn-irec.r-99.com/sites/default/files/imagecache/60i/pictures/15/picture-1579038-uXxH9EHj.jpg" src="/themes/irecommend_new/images/ajax-loader.gif" title="Информация о пользователе."/></a></div>
</div>
<div>
<div class="starsRating"><div class="fivestarWidgetStatic fivestarWidgetStatic-vote fivestarWidgetStatic-5"><div class="star"><div class="on"></div></div><div class="star"><div class="on"></div></div><div class="star"><div class="on"></div></div><div class="star"><div class="on"></div></div><div class="star"><div class="off"></div></div></div></div>

Ищем класс с комметарием

In [113]:
reviewTitle = views.find(attrs={'class':'reviewTitle'})
reviewTitle

<div class="reviewTitle"><a href="/content/dvoe-passazhirov-kosmicheskogo-korablya-prosypayutsya-ranshe-polozhennogo-na-90-let-romantic">Двое пассажиров космического корабля, просыпаются раньше положенного на 90 лет. Романтическая история на космическом корабле </a></div>

Извлекаем комментарий

In [114]:
comment = reviewTitle.a.string
print(comment)

Двое пассажиров космического корабля, просыпаются раньше положенного на 90 лет. Романтическая история на космическом корабле 


Ищем класс с рейтингом

In [115]:
starsRating = views.find(attrs={'class':'starsRating'})
starsRating

<div class="starsRating"><div class="fivestarWidgetStatic fivestarWidgetStatic-vote fivestarWidgetStatic-5"><div class="star"><div class="on"></div></div><div class="star"><div class="on"></div></div><div class="star"><div class="on"></div></div><div class="star"><div class="on"></div></div><div class="star"><div class="off"></div></div></div></div>

Подсчитываем количество звезд (поставлена - class='on', нет - class='off')

In [116]:
rating = len(starsRating.findAll(attrs={'class':'on'}))
rating

4

Пишем функцию

In [117]:
def getText(body):
    """
        Возвращает комментарий и оценку
    """
    reviewTitle = body.find(attrs={'class':'reviewTitle'})
    comment = reviewTitle.a.string
    
    #if comment:
    #    comment = comment.split('\n')
    #    comment = " ".join(comment).strip()
    #else:
    #    comment = ""
    
    starsRating = body.find(attrs={'class':'starsRating'})
    
    rating = len(starsRating.findAll(attrs={'class':'on'}))

    return {'Comment': comment, 'Rating':rating}    

In [118]:
getText(views)

{'Comment': 'Двое пассажиров космического корабля, просыпаются раньше положенного на 90 лет. Романтическая история на космическом корабле ',
 'Rating': 4}

In [119]:
def getPageData(url, page_number):
    # составляем ссылку на страницу поиска
    if page_number == 0:
        page_link = url
    else:
        page_link = '{}?page={}'.format(url,page_number)
    
    print(page_link)

    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})

    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return []

    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    return soup.findAll(attrs={'class':'smTeaser'})

In [120]:
len(getPageData('https://irecommend.ru/content/passazhiry-passengers-0', 4))

https://irecommend.ru/content/passazhiry-passengers-0?page=4


36

In [121]:
# Функция, которая собирает данные и сохраняет в DataFrame
sleep = 15 
def appendData(url, df):
    for page_number in range(1000):
        list = getPageData(url, page_number)

        for body in list:
            data_row = getText(body)
            df = df.append(data_row, ignore_index=True).dropna(axis = 1)

        print(len(df))    
        # По 50 комментариев на страницу, если меньше, то завершаем, после обработки
        if len(list) < 50:
            break
        time.sleep(sleep)    
    return df        

In [37]:
url = 'https://irecommend.ru/content/passazhiry-passengers-0'
df = pd.DataFrame(columns=['Comment',  'Rating'])
df = appendData(url, df)

https://irecommend.ru/content/passazhiry-passengers-0
https://irecommend.ru/content/passazhiry-passengers-0?page=1
https://irecommend.ru/content/passazhiry-passengers-0?page=2
https://irecommend.ru/content/passazhiry-passengers-0?page=3


In [38]:
df

,Comment,Rating
0,"""Перестань волноваться о том, что не можешь ко...",5
1,"Двое пассажиров космического корабля, просыпаю...",4
2,"Давно не видела таких бредовых фильмов. ""Пасса...",1
3,"Человеку нужен человек. Если нельзя, но очень ...",5
4,Неплохая вариация на тему космоса и одиночества,4
...,...,...
181,Однозначно понравилось!! Давно так не было!,5
182,"Можно чуть меньше романтики было, а в целом ко...",4
183,"Яркий, но одноразовый",4
184,Нудятина,2


### Собираем больше данных (по нескольким фильмам)

In [225]:
pages = [
    'https://irecommend.ru/content/passazhiry-passengers-0',
    'https://irecommend.ru/content/zelenaya-milya',
    'https://irecommend.ru/content/2012',
    'https://irecommend.ru/content/neprikasaemye-intouchables',
    'https://irecommend.ru/content/ivan-vasilevich-menyaet-professiyu',
    'https://irecommend.ru/content/dzhoker-0',
    'https://irecommend.ru/content/kruella-cruella-2021-film',
    'https://irecommend.ru/content/gordost-i-predubezhdenie',
    'https://irecommend.ru/content/my-millery',
    'https://irecommend.ru/content/vremya-time',
    'https://irecommend.ru/content/bogemskaya-rapsodiya',
    'https://irecommend.ru/content/dyavol-nosit-prada-0',
    'https://irecommend.ru/content/t-34',
    'https://irecommend.ru/content/interstellar',
    'https://irecommend.ru/content/dostat-nozhi-2019-film',
    'https://irecommend.ru/content/tri-metra-nad-nebom',
    'https://irecommend.ru/content/volk-s-uoll-strit-2013'
]

In [171]:
# Добавлял данные, чтобы увеличить объем данных
pages = [
    #'https://irecommend.ru/content/sinister',
    #'https://irecommend.ru/content/vremya-old-2021-film',
    #'https://irecommend.ru/content/dukhless',
    #'https://irecommend.ru/content/anna-karenina-2012g',
    #'https://irecommend.ru/content/sumerki',
    #'https://irecommend.ru/content/devushka-v-poezde-girl-train',
    #'https://irecommend.ru/content/moskva-slezam-ne-verit',
    #'https://irecommend.ru/content/vek-adelin',
    #'https://irecommend.ru/content/begushchii-v-labirinte-maze-runner',
    #'https://irecommend.ru/content/zarnitsa-2021-film',
    #'https://irecommend.ru/content/fantasticheskie-tvari-tainy-dambldora-fantastic-beasts-secrets-dumbledore-2022-film',
    #'https://irecommend.ru/content/odnoi-bezumnoi-nochyu-2023-film',
    #'https://irecommend.ru/content/epitsentr-buri-2023-film',
    #'https://irecommend.ru/content/eskortnitsa-2023-film',
    #'https://irecommend.ru/content/uglovaya-komnata-1962-film',
    #'https://irecommend.ru/content/v-efire-line-2022-film',
    #'https://irecommend.ru/content/smotret-v-obe-storony-look-both-ways-2022-film',
    #'https://irecommend.ru/content/yura-dvornik-2023-film',
    #'https://irecommend.ru/content/anon-anon-0',
    #'https://irecommend.ru/content/za-palycha-2023-film',
    #'https://irecommend.ru/content/raiskoe-shosse-2022-film',
    #'https://irecommend.ru/content/illyuziya-pobega-2023-film',
    #'https://irecommend.ru/content/skandal-2019-film-0',
    #'https://irecommend.ru/content/proniknovenie-2022-film',
    #'https://irecommend.ru/content/cheburashka-2022-film-0',
    #'https://irecommend.ru/content/groza-2020-film',
    #'https://irecommend.ru/content/13-minut-2021-film',
    #'https://irecommend.ru/content/temnyi-mir',
    #'https://irecommend.ru/content/malefisenta-vladychitsa-tmy-maleficent-mistress-evil'
]

In [172]:
full_df = pd.DataFrame(columns=['Comment',  'Rating'])
for url in pages:
    full_df = appendData(url, full_df)
    time.sleep(sleep)  

https://irecommend.ru/content/illyuziya-pobega-2023-film
9
https://irecommend.ru/content/skandal-2019-film-0
26
https://irecommend.ru/content/proniknovenie-2022-film
31
https://irecommend.ru/content/cheburashka-2022-film-0
81
https://irecommend.ru/content/cheburashka-2022-film-0?page=1
131
https://irecommend.ru/content/cheburashka-2022-film-0?page=2
181
https://irecommend.ru/content/cheburashka-2022-film-0?page=3
231
https://irecommend.ru/content/cheburashka-2022-film-0?page=4
281
https://irecommend.ru/content/cheburashka-2022-film-0?page=5
331
https://irecommend.ru/content/cheburashka-2022-film-0?page=6
381
https://irecommend.ru/content/cheburashka-2022-film-0?page=7
386
https://irecommend.ru/content/groza-2020-film
396
https://irecommend.ru/content/13-minut-2021-film
405
https://irecommend.ru/content/temnyi-mir
434
https://irecommend.ru/content/malefisenta-vladychitsa-tmy-maleficent-mistress-evil
484
https://irecommend.ru/content/malefisenta-vladychitsa-tmy-maleficent-mistress-evil?p

In [217]:
full_df

,Comment,Rating
0,"""Перестань волноваться о том, что не можешь ко...",5
1,"Двое пассажиров космического корабля, просыпаю...",4
2,"Давно не видела таких бредовых фильмов. ""Пасса...",1
3,"Человеку нужен человек. Если нельзя, но очень ...",5
4,Неплохая вариация на тему космоса и одиночества,4
...,...,...
6742,"Фильм, который стоит посмотреть!",5
6743,Завораживающий - breathtaking!,5
6744,"Малефисента-2 - сиквел удался, или мои впечатл...",4
6745,Вау!,5


In [176]:
full_df['Comment'] = full_df['Comment'].astype( str )
full_df['Rating'] = full_df['Rating'].astype(int)

Сохраняем данные в файл, чтобы не потерять

In [179]:
pd.to_pickle(full_df, "data.pkl")

In [177]:
df = pd.read_pickle("data.pkl")
df

,Comment,Rating
0,"""Перестань волноваться о том, что не можешь ко...",5
1,"Двое пассажиров космического корабля, просыпаю...",4
2,"Давно не видела таких бредовых фильмов. ""Пасса...",1
3,"Человеку нужен человек. Если нельзя, но очень ...",5
4,Неплохая вариация на тему космоса и одиночества,4
...,...,...
6183,"Когда две реальности сливаются в одну, появляе...",3
6184,"Трое вышли из леса, зайдя в него вшестером.",4
6185,«Иллюзия побега» - триллер о кризисе в отношен...,4
6186,"""Я хочу найти тебя и себя""... Куда бы ты не бе...",4


In [515]:
#full_df = df.append(full_df, ignore_index=True)
#full_df

# Работа с текстом

In [180]:
%matplotlib inline
#! pip install pymorphy2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from sklearn.metrics import *
import warnings
warnings.filterwarnings("ignore")

Преобразуем dataframe в json

In [181]:
data = full_df.to_json(orient='records')
data = json.loads(data)

In [182]:
data[0]

{'Comment': '"Перестань волноваться о том, что не можешь контролировать, просто живи" - пожалуй, это самый лучший жизненный совет|Робинзонада в космосе✨🌙|Один из моих любимых фильмов. Когда мне грустно, я включаю "Пассажиры".',
 'Rating': 5}

In [183]:
from collections import Counter
print(len(data))
Counter([x['Rating'] for x in data]).most_common()

6747


[(5, 4469), (4, 1066), (3, 609), (2, 329), (1, 274)]

### Предобработка данных

Удаляем стоп-слова, пунктуацию, смайлики и т.д.

In [184]:
import nltk   # Natural Language Toolkit
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

word_tokenizer = nltk.WordPunctTokenizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tima-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [185]:
import re
regex = re.compile(r'[А-Яа-яA-zёЁ]+')


def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text)).lower()
    except:
        return ""

def process_data(data):
    texts = []
    targets = []

    # поочередно проходим по всем новостям в списке
    for item in tqdm(data):

        text_lower = words_only(item['Comment']) # оставим только слова
        tokens     = word_tokenizer.tokenize(text_lower) #разбиваем текст на слова

        # удаляем пунктуацию и стоп-слова
        tokens = [word for word in tokens if (word not in stop_words and not word.isnumeric())]

        texts.append(tokens) # добавляем в предобработанный список

    return texts

In [399]:
y = [item['Rating'] for item in data]
texts = process_data(data)


100%|███████████████████████████████████████████████████████████████████████████| 6747/6747 [00:00<00:00, 53224.06it/s]


In [400]:
# example
i = 1
print("Rating: ", y[i])
print("Comment: ", texts[i])

Rating:  4
Comment:  ['двое', 'пассажиров', 'космического', 'корабля', 'просыпаются', 'раньше', 'положенного', 'лет', 'романтическая', 'история', 'космическом', 'корабле']


### Нормализация слов

In [401]:
# загружаем библиотеку для лемматизации
import pymorphy2 # Морфологический анализатор

# инициализируем лемматизатор :)
morph = pymorphy2.MorphAnalyzer()

In [402]:
i = 1
for aword in texts[i]:
    aword_norm = morph.parse(aword)[0].normal_form
    print("Исходное слово: %s\tЛемматизированное: %s" % (aword, aword_norm))

Исходное слово: двое	Лемматизированное: двое
Исходное слово: пассажиров	Лемматизированное: пассажир
Исходное слово: космического	Лемматизированное: космический
Исходное слово: корабля	Лемматизированное: корабль
Исходное слово: просыпаются	Лемматизированное: просыпаться
Исходное слово: раньше	Лемматизированное: ранний
Исходное слово: положенного	Лемматизированное: положить
Исходное слово: лет	Лемматизированное: год
Исходное слово: романтическая	Лемматизированное: романтический
Исходное слово: история	Лемматизированное: история
Исходное слово: космическом	Лемматизированное: космический
Исходное слово: корабле	Лемматизированное: корабль


In [403]:
# применяем лемматизацию ко всем текстам
texts_normal = texts.copy()
for i in range(len(texts)):           # tqdm_notebook создает шкалу прогресса :)
    text_lemmatized = [morph.parse(x)[0].normal_form for x in texts[i]] # применяем лемматизацию для каждого слова в тексте
    texts_normal[i] = ' '.join(text_lemmatized)                # объединяем все слова в одну строку через пробел

In [404]:
# example
i = 1
print("Rating: ", y[i])
print("Comment: ", texts_normal[i])

Rating:  4
Comment:  двое пассажир космический корабль просыпаться ранний положить год романтический история космический корабль


<b>Преобразуем целевую переменную</b>

```
Где оценка 1-3 - это негативные отзывы
Где оценка 4-5 - это положительные отзывы
```

In [422]:
# Функция для кодирования лейблов
def num2boolean(y):
    if y == 1 or y == 2 or y == 3:
        return 0
    if y == 4 or y == 5:
        return 1

y = [num2boolean(yy) for yy in y]

In [423]:
Counter(y)

Counter({1: 5535, 0: 1212})

Разбиваем данные

In [424]:
#train test_split
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_y, test_y = train_test_split(texts_normal, y, test_size=0.20, random_state=42, stratify = y)

### TF-IDF

In [425]:
#вычисляем tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
# Fit TF-IDF on train texts
vectorizer = TfidfVectorizer(max_features = 200, norm = None) # возмем топ 200 слов
vectorizer.fit(train_texts)

# Топ-10 слов
vectorizer.get_feature_names()[:10]

['актёр',
 'американский',
 'бежать',
 'быть',
 'ваш',
 'век',
 'великий',
 'великолепный',
 'весь',
 'вечный']

In [426]:
# Обучаем TF-IDF на train, а затем применяем к train и test
train_X = vectorizer.fit_transform(train_texts)
test_X  = vectorizer.transform(test_texts)

### Обучаем классификатор:  Случайный лес (RandomForest)

In [427]:
#import алгоритма из библиотеки
from sklearn.ensemble import RandomForestClassifier

# инициализируем модель
clf = RandomForestClassifier(n_estimators = 1500, max_depth = 50)

# обучаем ее на тренировочных данных
clf = clf.fit(train_X, train_y)

# делаем предсказание для тестовых данных
pred = clf.predict(test_X)
pred_proba = clf.predict_proba(test_X)

In [428]:
print('Accuracy: ', accuracy_score(test_y, pred))
print('F1: ', f1_score(test_y, pred, average = 'macro'))

Accuracy:  0.8244444444444444
F1:  0.513945118866586


In [430]:
print(classification_report(test_y, pred))

              precision    recall  f1-score   support

           0       0.61      0.07      0.13       243
           1       0.83      0.99      0.90      1107

    accuracy                           0.82      1350
   macro avg       0.72      0.53      0.51      1350
weighted avg       0.79      0.82      0.76      1350



In [442]:
roc_auc_score(test_y, pred_proba[:,1], average='macro')

0.7490604124148239

### Стохастический градиентый спуск

In [443]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(alpha=.00001, max_iter=300, penalty="l1", loss="log", shuffle=True, random_state=42)

# обучаем ее на тренировочных данных
clf = clf.fit(train_X, train_y)

# делаем предсказание для тестовых данных
pred = clf.predict(test_X)
pred_proba = clf.predict_proba(test_X)

In [444]:
print('Accuracy: ', accuracy_score(test_y, pred))
print('F1: ', f1_score(test_y, pred, average = 'macro'))

Accuracy:  0.7896296296296297
F1:  0.5511820978549455


In [445]:
print(classification_report(test_y, pred))

              precision    recall  f1-score   support

           0       0.33      0.17      0.22       243
           1       0.84      0.93      0.88      1107

    accuracy                           0.79      1350
   macro avg       0.58      0.55      0.55      1350
weighted avg       0.75      0.79      0.76      1350



In [446]:
roc_auc_score(test_y, pred_proba[:,1], average='macro')

0.6654343292404118

### Метод Опорных Векторов (SVM)

In [447]:
from sklearn.svm import SVC
clf = SVC(probability=True)
# обучаем ее на тренировочных данных
clf = clf.fit(train_X, train_y)

# делаем предсказание для тестовых данных
pred = clf.predict(test_X)
pred_proba = clf.predict_proba(test_X)

In [448]:
print('Accuracy: ', accuracy_score(test_y, pred))
print('F1: ', f1_score(test_y, pred, average = 'macro'))

Accuracy:  0.8244444444444444
F1:  0.48682785997834715


In [481]:
print(classification_report(test_y, pred))

              precision    recall  f1-score   support

           0       0.30      0.77      0.43       243
           1       0.92      0.60      0.73      1107

    accuracy                           0.63      1350
   macro avg       0.61      0.69      0.58      1350
weighted avg       0.81      0.63      0.68      1350



In [449]:
roc_auc_score(test_y, pred_proba[:,1], average='macro')

0.5770610518176512

### Выводы

Были рассмотрены три классификатора

Точность:

    RandomForest - 0.8244444444444444
    SVN - 0.8244444444444444
    SGD - 0.7896296296296297

Но у RandomForest - лучше ROC (0.7490604124148239), чем у SVN (0.5770610518176512)